# Discovering a Process Model using Next Event Predictors

In this notebook we discover a process model using the trained next event predictor.

* *Event Log*: The event log for which the process model should be discovered.
* *Model*: The next event predictor to use. The event log the next event predictor was trained on must match the chosen event log.
* *Automatic Threshold*: If checked, the next event threshold is determined automatically.
* *Threshold*: The next event threshold to use when no automatic threshold is used. Might fail.
* *Use Cache*: Whether to use a cache when generating cases.
* *Group Attribute Nodes*: Whether to also group attribute nodes in the process model.
* *Use Event Attributes*: Whether to also discover event attributes. Must match the used next event predictor.

In [1]:
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))

from ipywidgets import widgets, interact, interact_manual, Layout, Button, Box
from IPython.display import display

from april.fs import DATE_FORMAT, EVENTLOG_DIR, MODEL_DIR
from r2pa.api import routines

import arrow
import os


event_log_widget = widgets.Dropdown(description='Event Log')
model_name_widget = widgets.Dropdown(description='Model')
next_event_threshold_widget = widgets.FloatSlider(description='Threshold', min=0, max=1, step=0.01, value=0.1)
use_cache_widget = widgets.Checkbox(description='Use Cache', value=True)
automatic_threshold_widget = widgets.Checkbox(description='Automatic Threshold', value=False)
group_attributes_widget = widgets.Checkbox(description='Group Attribute Nodes', value=False)
event_attributes_widget = widgets.Checkbox(description='Use Event Attributes', value=True)

reload_button = widgets.Button(description="Reload")
discover_button = widgets.Button(description="Discover")

first_row = widgets.HBox([event_log_widget, model_name_widget, reload_button])
second_row = widgets.HBox([automatic_threshold_widget, next_event_threshold_widget])
third_row = widgets.HBox([use_cache_widget, group_attributes_widget, event_attributes_widget])
fourth_row = widgets.HBox([discover_button])

parameter_gui = widgets.Output()
output = widgets.Output()

with parameter_gui:
    display(widgets.VBox([first_row, second_row, third_row, fourth_row]))

def get_all_event_logs():
    files = os.listdir(EVENTLOG_DIR)
    return [file[0:len(file)-8] for file in files if file[-8:] == '.json.gz']

def get_all_models():
    files = os.listdir(MODEL_DIR)
    return [file[0:len(file)-3] for file in files if file[-3:] == '.h5']

def on_value_change(change):
    checkbox_value = change['new']
    next_event_threshold_widget.disabled = True if checkbox_value else False
    next_event_threshold_widget.layout.visibility = 'hidden' if checkbox_value else 'visible'
    

def on_reload(button):
    event_log_widget.options = get_all_event_logs()
    model_name_widget.options = get_all_models()

automatic_threshold_widget.observe(on_value_change, names='value')

def discover_process_model(button):
    event_log, model, next_event_threshold = event_log_widget.value, model_name_widget.value, -1 if automatic_threshold_widget.value else next_event_threshold_widget.value 
    use_cache, group_attribute_nodes, use_event_attributes = use_cache_widget.value, group_attributes_widget.value, event_attributes_widget.value
    
    start_time = arrow.now()
    
    output_name = f'results_{event_log}-{model}-{start_time.format(DATE_FORMAT)}-{use_cache}-{group_attribute_nodes}-{next_event_threshold}'
    output_locations = [output_name]
    
    output.clear_output()
    with output:
        print(f'The discovered process model will be stored as {output_name}')
        routines.discovery(output_locations=output_locations, event_log=event_log, model=model, next_event_threshold=next_event_threshold, use_cache=use_cache, group_attribute_nodes=group_attribute_nodes)


discover_button.on_click(discover_process_model)
reload_button.on_click(on_reload)
on_reload(None)

display(parameter_gui)
display(output)

Output()

Output()